In [1]:
import time
import torch

from net import Net
from aco import ACO
from utils import load_test_dataset
from tqdm import tqdm

# torch.manual_seed(12345)

EPS = 1e-10
device = 'cpu'

In [2]:
@torch.no_grad()
def infer_instance(model, pyg_data, distances, n_ants, t_aco_diff, k_sparse=None):
    if model:
        model.eval()
        heu_vec = model(pyg_data)
        heu_mat = model.reshape(pyg_data, heu_vec) + EPS
    
        aco = ACO(
            n_ants=n_ants,
            heuristic=heu_mat.cpu(),
            distances=distances.cpu(),
            device='cpu',
            two_opt = True,
            # elitist=True
            # min_max = True
        )
    
    else:
        aco = ACO(
            n_ants=n_ants,
            distances=distances.cpu(),
            device='cpu',
            two_opt = True,
            # elitist=True
            # min_max = True
        )
        # if k_sparse:
        #     aco.sparsify(k_sparse)
        
    results = torch.zeros(size=(len(t_aco_diff),))
    for i, t in enumerate(t_aco_diff):
        best_cost = aco.run(t, inference = True)
        results[i] = best_cost
    return results
        
    
@torch.no_grad()
def test(dataset, model, n_ants, t_aco, k_sparse=None):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),))
    start = time.time()
    for pyg_data, distances in tqdm(dataset):
        results = infer_instance(model, pyg_data, distances, n_ants, t_aco_diff, k_sparse)
        sum_results += results
    end = time.time()
    
    return sum_results / len(dataset), end-start

In [10]:
def compare(n_node, k_sparse = None, n_ants=30, t_aco = None):
    global device
    k_sparse = k_sparse or max(10, n_node//10)
    t_aco = None or [1, 2, 5, 10, 20, 30, 50]
    device = 'cpu' if n_node < 200 else 'cuda:0'
    test_list = load_test_dataset(n_node, k_sparse, device, start_node = 0)
    print("number of instances:", len(test_list))
    print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

    print("=== MetaACO ===")
    net_tsp = Net().to(device)
    net_tsp.load_state_dict(torch.load(f'../pretrained/tsp_2opt/tsp{n_node}.pt', map_location=device))
    avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco, k_sparse)
    print('total duration: ', duration)
    for i, t in enumerate(t_aco):
        print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

    return
    print("=== ACO ===")
    avg_aco_best, duration = test(test_list, None, n_ants, t_aco, k_sparse)
    print('total duration: ', duration)
    for i, t in enumerate(t_aco):
        print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

In [11]:
compare(50)

number of instances: 1280
device: cpu
=== MetaACO ===


100%|██████████| 1280/1280 [07:58<00:00,  2.68it/s]

total duration:  478.06916213035583
T=1, average cost is 5.713399410247803.
T=2, average cost is 5.6996870040893555.
T=5, average cost is 5.692080497741699.
T=10, average cost is 5.690206527709961.
T=20, average cost is 5.689507484436035.
T=30, average cost is 5.689334392547607.
T=50, average cost is 5.6890459060668945.


In [8]:
compare(500)

number of instances: 128
device: cuda:0+cpu
=== MetaACO ===


100%|██████████| 128/128 [24:07<00:00, 11.31s/it]

total duration:  1447.8868181705475
T=1, average cost is 17.062171936035156.
T=2, average cost is 17.011945724487305.
T=5, average cost is 16.94670295715332.
T=10, average cost is 16.90425682067871.
T=20, average cost is 16.87262725830078.
T=30, average cost is 16.85939598083496.
T=50, average cost is 16.845340728759766.
